In [43]:
# -*- coding: UTF-8 -*-
import sys
import time
import leveldb
from urllib import quote_plus
import pandas as pd
import numpy as np
import re
import json
import itertools
import sys
import requests

reload(sys)
sys.setdefaultencoding('utf-8')
def url_get(url):
    # print('GET ' + url)
    header = dict()
    header['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
    header['Accept-Encoding'] = 'gzip,deflate,sdch'
    header['Accept-Language'] = 'en-US,en;q=0.8'
    header['Connection'] = 'keep-alive'
    header['DNT'] = '1'
    # header['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.71 Safari/537.36'
    header['User-Agent'] = 'Mozilla/12.0 (compatible; MSIE 8.0; Windows NT)'
    return requests.get(url, timeout=5, headers=header).text

URL_BASE = 'http://api.s.m.taobao.com/search.json?m=shopitemsearch&sellerId={}&n=40&page={}'
item_page = '1'
sellerId = '4135951433'
sellerId = '2471843743'
df = pd.DataFrame(columns=['id', 'title', 'price', 'priceWithRate', 'isprepay'])
while True:
    url = URL_BASE.format(quote_plus(sellerId), item_page)
    for tr in range(5):
        try:
            items_obj = json.loads(url_get(url))
            break
        except KeyboardInterrupt:
            quit()
        except Exception as e:
            if tr == 4: raise e
     
    if len(items_obj['itemsArray']) == 0:
        print('no listItem')
        break
        
    for item in items_obj['itemsArray']:       
        df = df.append({
            'id': item['item_id'],
            'title': item['title'],
            'price': item['price'],
            'priceWithRate': item['priceWithRate'],
            'isprepay': int(item['isprepay']) if item.has_key('isprepay') else 0
        }, ignore_index=True)
               
    if int(items_obj['totalPage']) > int(item_page):
        item_page = str(int(item_page) + 1)
    else:
        break





In [44]:

items = [u'弹力美肌水', u'三剑客', u'薰衣草', u'眼霜']
items_df = []
d = {}
for i, item in enumerate(items):   
    df0 = df[df['title'].str.contains(item)]
    items_df.append(df0)
    
    items_index = df0.index.tolist()
    minPrice = float(df0['priceWithRate'].min())
    minPriceIndex = df0['priceWithRate'].astype(np.float64).idxmin()
    d[i] = {
        'items_index': items_index,
        'min_price': minPrice,
        'min_price_index': minPriceIndex
    }
print d

In [46]:
coupons = pd.DataFrame([
    {
        'condition': 3000,
        'discount': 300,
        'method': 'reach'
    },
    {
        'condition': 2000,
        'discount': 200,
        'method': 'reach'
    },
    {
        'condition': 1500,
        'discount': 150,
        'method': 'reach'
    },
    {
        'condition': 6,
        'discount': 5,
        'method': 'reach'
    }
])
